In [2]:
%matplotlib inline
import gzip
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pystan
import lda
import lda.datasets
import matplotlib.cm as cm
import cPickle as cpkl

In [4]:
#loading the preprocessed data. See data_processing.ipynb for details
with open('preprocessed_data.cpkl','r') as f:
    data = cpkl.load(f)
    
x_train = data['x_train']     #one hot matrix (samples x genes)
t_train = data['t_train']     #labels from bioinformatician 
genes_id = data['genes_id']   
genes = data['genes']
sampleid = data['sampleid']   #sampleid and words contain the same information as x_train just encoded as sampleid: cell, words: id for seen words  
words = data['words']



In [16]:


# ==============================================================================
LDA_uniprior = """

data {
  int<lower=2> K;               // num topics
  int<lower=2> V;               // num words
  int<lower=1> M;               // num docs
  int<lower=1> N;               // total word instances
  int<lower=1,upper=V> w[N];    // word n
  int<lower=1,upper=M> doc[N];  // doc ID for word n
  vector<lower=0>[K] alpha;     // topic prior
  vector<lower=0>[V] beta;      // word prior
}
parameters {
  simplex[K] theta[M];   // topic dist for doc m
  simplex[V] phi[K];     // word dist for topic k
}
model {
  for (m in 1:M)  
    theta[m] ~ dirichlet(alpha);  // prior
  for (k in 1:K)  
    phi[k] ~ dirichlet(beta);     // prior
  for (n in 1:N) {
    real gamma[K];
    for (k in 1:K) 
      gamma[k] <- log(theta[doc[n],k]) + log(phi[k,w[n]]);
    increment_log_prob(log_sum_exp(gamma));  // likelihood
  }
}
"""

M,V  = x_train.shape
K=5
V=48
M=1430
data = dict(K=K,
        V=V,
        M=M,
        N=np.sum(x_train),
        w=words,
        doc=sampleid,
        alpha=np.ones(K,)*(1./K),
        beta=np.ones(V,)*(1./V),
           )

fit_uniprior = pystan.stan(model_code=LDA_uniprior, data=data)


samples = fit.extract()

ValueError: anon_model_177a53e2209357cea25bf3efc5f97bd6_namespace::anon_model_177a53e2209357cea25bf3efc5f97bd6: w[k0__] is 79, but must be less than or equal to 48

In [12]:
LDA_informative_prior = """

data {
  int<lower=2> K;               // num topics
  int<lower=2> V;               // num words
  int<lower=1> M;               // num docs
  int<lower=1> N;               // total word instances
  int<lower=1,upper=V> w[N];    // word n
  int<lower=1,upper=M> doc[N];  // doc ID for word n
  vector<lower=0>[K] alpha;     // topic prior
  vector<lower=0>[V] beta;      // word prior
}
parameters {
  simplex[K] theta[M];   // topic dist for doc m
  simplex[V] phi[K];     // word dist for topic k
}
model {
  for (m in 1:M)  
    theta[m] ~ dirichlet(alpha);  // prior
  for (k in 1:K)  
    phi[k] ~ dirichlet(beta);     // prior
  for (n in 1:N) {
    real gamma[K];
    for (k in 1:K) 
      gamma[k] <- log(theta[doc[n],k]) + log(phi[k,w[n]]);
    increment_log_prob(log_sum_exp(gamma));  // likelihood
  }
}
"""


M,V  = x_train.shape
K=5
V=48
M=1430
data = dict(K=K,
        V=V,
        M=M,
        N=np.sum(x_train),
        w=words,
        doc=sampleid,
        alpha=np.ones(K,)*(1./K),
        beta=(1+x_train.sum(axis=1).astype('float32'))/x_train.sum(),
           )

fit_uniprior = pystan.stan(model_code=LDA_uniprior, data=data)



48

[9,
 79,
 85,
 110,
 116,
 134,
 154,
 210,
 210,
 247,
 275,
 339,
 344,
 391,
 391,
 391,
 391,
 417,
 508,
 701,
 712,
 733,
 737,
 796,
 810,
 856,
 867,
 917,
 1026,
 1092,
 1117,
 1154,
 1208,
 1218,
 1222,
 1244,
 1271,
 1297,
 1356,
 1390,
 800,
 3,
 7,
 8,
 28,
 30,
 37,
 37,
 39,
 43,
 47,
 48,
 54,
 54,
 55,
 55,
 55,
 56,
 57,
 58,
 62,
 64,
 75,
 77,
 82,
 82,
 92,
 104,
 106,
 108,
 110,
 114,
 114,
 114,
 114,
 117,
 121,
 126,
 130,
 130,
 130,
 130,
 130,
 130,
 131,
 133,
 134,
 134,
 134,
 134,
 134,
 134,
 134,
 142,
 164,
 164,
 180,
 181,
 186,
 195,
 203,
 214,
 222,
 229,
 229,
 231,
 234,
 238,
 239,
 241,
 241,
 245,
 254,
 259,
 272,
 274,
 274,
 274,
 277,
 284,
 293,
 295,
 297,
 298,
 300,
 306,
 316,
 321,
 323,
 328,
 330,
 335,
 348,
 348,
 350,
 350,
 356,
 356,
 360,
 361,
 364,
 364,
 374,
 377,
 386,
 387,
 387,
 388,
 388,
 391,
 391,
 391,
 394,
 394,
 394,
 396,
 400,
 402,
 402,
 402,
 407,
 415,
 425,
 427,
 427,
 439,
 440,
 441,
 447,
 447,
 